In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score

%matplotlib inline

In [70]:
data = pd.read_csv("../data/CreditScoring.csv")

In [71]:
def data_prep_function(data):
    df = data.copy()
    df.columns = df.columns.map(lambda x: x.lower())
    
    status_map = {
    1: "ok",
    2: "default",
    0: "unknown"
    }
    df['status'] = df['status'].map(status_map)
    home_ownership_map = {
        1: "rent",
        2: "owner",
        3: "priv",
        4: "ignore",
        5: "parents",
        6: "other",
        0: "unknown"
    }
    df['home'] = df['home'].map(home_ownership_map)
    marital_status_map = {
        1: "single",
        2: "married",
        3: "widowed",
        4: "separated",
        5: "divorced",
        0: "unknown",
    }
    df['marital'] = df['marital'].map(marital_status_map)
    records_map = {
        1 : "no_records",
        2 : "yes_records"
    }
    df['records'] = df['records'].map(records_map)
    job_status_map = {
        1: "fixed",
        2: "part-time",
        3: "freelance",
        4: "others",
        0: "unknown"
    }
    df['job'] = df['job'].map(job_status_map)
    
    df['income'] = df['income'].replace(to_replace=99999999, value = np.nan)
    df['assets'] = df['assets'].replace(to_replace=99999999, value = np.nan)
    df['debt'] = df['debt'].replace(to_replace=99999999, value = np.nan)
    
    df = df[df['status'] != 'unknown'].reset_index(drop=True)
    
    y = (df['status'] == 'default').astype(int)
    X = df.drop(columns=['status'])
    
    X['income'] = X['income'].fillna(np.median(X['income'].dropna()))
    X['assets'] = X['assets'].fillna(0)
    X['debt'] = X['debt'].fillna(0)
    
    return X, y

In [72]:
X, y = data_prep_function(data)

In [73]:
X.isnull().sum()    

seniority    0
home         0
time         0
age          0
marital      0
records      0
job          0
expenses     0
income       0
assets       0
debt         0
amount       0
price        0
dtype: int64

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [75]:
dv = DictVectorizer(sparse=False)
X_train_dict = X_train.to_dict(orient='records')
X_train_final = dv.fit_transform(X_train_dict)


In [76]:
X_train_final

array([[3.80e+01, 8.50e+02, 3.50e+03, ..., 0.00e+00, 1.00e+00, 2.40e+01],
       [4.80e+01, 8.50e+02, 5.00e+03, ..., 1.00e+00, 1.40e+01, 4.80e+01],
       [4.60e+01, 5.00e+02, 8.00e+03, ..., 0.00e+00, 1.20e+01, 3.60e+01],
       ...,
       [5.40e+01, 6.00e+02, 5.00e+03, ..., 0.00e+00, 2.00e+01, 6.00e+01],
       [3.80e+01, 3.00e+03, 1.25e+04, ..., 1.00e+00, 3.00e+00, 3.60e+01],
       [5.10e+01, 3.50e+02, 2.00e+03, ..., 0.00e+00, 1.20e+01, 1.20e+01]],
      shape=(3117, 27))

In [77]:
dt = DecisionTreeClassifier()
dt.fit(X_train_final, y_train)

DecisionTreeClassifier()

In [78]:
X_test_dict = X_test.to_dict(orient='records')
X_test_final = dv.transform(X_test_dict)
y_pred = dt.predict(X_test_final)

In [79]:
roc_auc_score(y_test, y_pred)

np.float64(0.6688099653309683)

In [80]:
for depths in [1,2,3,4,5,10,15,20]:
    dt = DecisionTreeClassifier(max_depth=depths)
    dt.fit(X_train_final, y_train)
    y_pred = dt.predict(X_test_final)
    print(f"Depth: {depths}, AUC: {roc_auc_score(y_test, y_pred)}")

Depth: 1, AUC: 0.6201752264237147
Depth: 2, AUC: 0.6009406326426402
Depth: 3, AUC: 0.6597422666559166
Depth: 4, AUC: 0.6745236367545487
Depth: 5, AUC: 0.6719100217689269
Depth: 10, AUC: 0.6629283237926309
Depth: 15, AUC: 0.6581929103173965
Depth: 20, AUC: 0.6561517374828671


In [81]:
for depths in [1,2,3,4,5,10,15,20]:
    for samples in [1,2,3,4,5,6,7,10,18,20]:
        dt = DecisionTreeClassifier(max_depth=depths, min_samples_leaf=samples)
        dt.fit(X_train_final, y_train)
        y_pred = dt.predict(X_test_final)
        print(f"Depth: {depths}, Min Samples: {samples}, AUC: {roc_auc_score(y_test, y_pred)}")

Depth: 1, Min Samples: 1, AUC: 0.6201752264237147
Depth: 1, Min Samples: 2, AUC: 0.6201752264237147
Depth: 1, Min Samples: 3, AUC: 0.6201752264237147
Depth: 1, Min Samples: 4, AUC: 0.6201752264237147
Depth: 1, Min Samples: 5, AUC: 0.6201752264237147
Depth: 1, Min Samples: 6, AUC: 0.6201752264237147
Depth: 1, Min Samples: 7, AUC: 0.6201752264237147
Depth: 1, Min Samples: 10, AUC: 0.6201752264237147
Depth: 1, Min Samples: 18, AUC: 0.6201752264237147
Depth: 1, Min Samples: 20, AUC: 0.6201752264237147
Depth: 2, Min Samples: 1, AUC: 0.6009406326426402
Depth: 2, Min Samples: 2, AUC: 0.6009406326426402
Depth: 2, Min Samples: 3, AUC: 0.6009406326426402
Depth: 2, Min Samples: 4, AUC: 0.6009406326426402
Depth: 2, Min Samples: 5, AUC: 0.6009406326426402
Depth: 2, Min Samples: 6, AUC: 0.6009406326426402
Depth: 2, Min Samples: 7, AUC: 0.6009406326426402
Depth: 2, Min Samples: 10, AUC: 0.6009406326426402
Depth: 2, Min Samples: 18, AUC: 0.6009406326426402
Depth: 2, Min Samples: 20, AUC: 0.60094063264

In [82]:
dt = DecisionTreeClassifier(max_depth=10, min_samples_leaf=20)
dt.fit(X_train_final, y_train)
y_pred = dt.predict(X_test_final)
roc_auc_score(y_test, y_pred)

np.float64(0.6912252412050848)

In [83]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.89      0.85       942
           1       0.66      0.49      0.56       395

    accuracy                           0.77      1337
   macro avg       0.73      0.69      0.70      1337
weighted avg       0.76      0.77      0.76      1337



In [84]:
from sklearn.ensemble import RandomForestClassifier

In [86]:
rf = RandomForestClassifier(n_estimators=20)
rf.fit(X_train_final, y_train)
y_pred = rf.predict(X_test_final)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.94      0.86       942
           1       0.76      0.44      0.56       395

    accuracy                           0.79      1337
   macro avg       0.78      0.69      0.71      1337
weighted avg       0.79      0.79      0.77      1337



In [102]:
features = list(dv.get_feature_names_out())

In [107]:
dtrain = xgb.DMatrix(X_train_final, label=y_train, feature_names=features)
dtest = xgb.DMatrix(X_test_final, label=y_test, feature_names=features)
params = {
    'eta': 0.3,
    'max_depth': 10,
    'min_child_weight': 1,
    'objective': 'binary:logistic',
    'seed':1,
    'verbosity':1
}

In [113]:
model = xgb.train(params=params, dtrain=dtrain, num_boost_round=100)

In [114]:
y_pred = model.predict(dtest)
print(classification_report(y_test, y_pred > 0.5))

              precision    recall  f1-score   support

           0       0.81      0.91      0.86       942
           1       0.69      0.50      0.58       395

    accuracy                           0.79      1337
   macro avg       0.75      0.71      0.72      1337
weighted avg       0.78      0.79      0.78      1337



In [115]:
roc_auc_score(y_test, y_pred)

np.float64(0.8299524308635008)